In [ ]:
#NOVEL ALGORITHM FOR APT DETECTION

In [ ]:
!pip install lightgbm scikit-learn pandas torch numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# Install dependencies (uncomment if running in Colab)
# !pip install lightgbm scikit-learn pandas torch numpy

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from lightgbm import LGBMClassifier
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# === STEP 1: Load Dataset ===
df = pd.read_csv("/content/new_data.csv")  # Ensure this file is present in your directory

In [ ]:

# === STEP 2: Feature Split ===
temporal_cols = [
    'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min',
    'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min',
    'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min'
]

stat_graph_cols = [col for col in df.columns if col not in temporal_cols + ['Label']]
X_temporal = df[temporal_cols].values.reshape(-1, len(temporal_cols), 1)
X_statgraph = df[stat_graph_cols].values
y = df['Label'].values

X_train_t, X_test_t, X_train_s, X_test_s, y_train, y_test = train_test_split(
    X_temporal, X_statgraph, y, test_size=0.2, random_state=42
)

In [ ]:
# === STEP 3: Train LightGBM ===
lgb = LGBMClassifier()
lgb.fit(X_train_s, y_train)
lgb_train_out = lgb.predict_proba(X_train_s)
lgb_test_out = lgb.predict_proba(X_test_s)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.007384 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 9174
[LightGBM] [Info] Number of data points in the train set: 5750, number of used features: 59
[LightGBM] [Info] Start training from score -2.519228
[LightGBM] [Info] Start training from score -3.844771
[LightGBM] [Info] Start training from score -2.117369
[LightGBM] [Info] Start training from score -1.290510
[LightGBM] [Info] Start training from score -1.054554
[LightGBM] [Info] Start training from score -1.869110
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [W

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# === STEP 4: Define Models ===
import torch.nn as nn
import torch.nn.functional as F

class BiGRUModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, dropout_prob=0.3):
        super(BiGRUModel, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, batch_first=True, bidirectional=True)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size * 2, 64)

    def forward(self, x):
        _, h = self.gru(x)
        h = h.permute(1, 0, 2).reshape(x.size(0), -1)
        h = self.dropout(h)
        return self.fc(h)

class CNNModel(nn.Module):
    def __init__(self, dropout_prob=0.3):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, 3, padding=1)
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(32, 32)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.dropout(x)
        return self.fc(x)

num_classes = len(np.unique(y_train))  # Assuming y_train is a NumPy array

class FusionModel(nn.Module):
    def __init__(self):
        super(FusionModel, self).__init__()
        self.fc1 = nn.Linear(102, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, gru_out, cnn_out, lgb_out):
        x = torch.cat([gru_out, cnn_out, lgb_out], dim=1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
import time
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, log_loss

# === STEP 5: Train Hybrid Deep Ensemble with Early Stopping and Metrics ===

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gru_model = BiGRUModel().to(device)
cnn_model = CNNModel().to(device)
fusion_model = FusionModel().to(device)

train_t = torch.tensor(X_train_t, dtype=torch.float32).to(device)
train_cnn = train_t.view(train_t.size(0), 1, -1)
train_lgb = torch.tensor(lgb_train_out, dtype=torch.float32).to(device)
train_labels = torch.tensor(y_train, dtype=torch.long).to(device)

dataset = TensorDataset(train_t, train_cnn, train_lgb, train_labels)
loader = DataLoader(dataset, batch_size=64, shuffle=True)

criterion = nn.CrossEntropyLoss()  # add class weights here if needed
optimizer = torch.optim.Adam(list(gru_model.parameters()) +
                             list(cnn_model.parameters()) +
                             list(fusion_model.parameters()), lr=0.001)

patience = 5
best_val_loss = float('inf')
counter = 0

start_train_time = time.time()
for epoch in range(50):  # max epochs
    gru_model.train()
    cnn_model.train()
    fusion_model.train()

    total_loss = 0
    for x_gru, x_cnn, x_lgb, y in loader:
        optimizer.zero_grad()
        out_gru = gru_model(x_gru)
        out_cnn = cnn_model(x_cnn)
        out = fusion_model(out_gru, out_cnn, x_lgb)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_train_loss = total_loss / len(loader)

    gru_model.eval()
    cnn_model.eval()
    fusion_model.eval()

    with torch.no_grad():
        val_t = torch.tensor(X_test_t, dtype=torch.float32).to(device)
        val_cnn = val_t.view(val_t.size(0), 1, -1)
        val_lgb = torch.tensor(lgb_test_out, dtype=torch.float32).to(device)
        val_labels = torch.tensor(y_test, dtype=torch.long).to(device)

        out_gru = gru_model(val_t)
        out_cnn = cnn_model(val_cnn)
        val_out = fusion_model(out_gru, out_cnn, val_lgb)
        val_loss = criterion(val_out, val_labels).item()

    print(f"Epoch {epoch+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {val_loss:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
        torch.save({
            'gru': gru_model.state_dict(),
            'cnn': cnn_model.state_dict(),
            'fusion': fusion_model.state_dict()
        }, 'best_model.pth')
    else:
        counter += 1
        if counter >= patience:
            print("Early stopping triggered.")
            break
end_train_time = time.time()
train_duration = end_train_time - start_train_time

# Load best model
checkpoint = torch.load('best_model.pth')
gru_model.load_state_dict(checkpoint['gru'])
cnn_model.load_state_dict(checkpoint['cnn'])
fusion_model.load_state_dict(checkpoint['fusion'])

# === STEP 6: Evaluation with detailed metrics ===
gru_model.eval()
cnn_model.eval()
fusion_model.eval()

start_infer_time = time.time()
with torch.no_grad():
    test_t = torch.tensor(X_test_t, dtype=torch.float32).to(device)
    test_cnn = test_t.view(test_t.size(0), 1, -1)
    test_lgb = torch.tensor(lgb_test_out, dtype=torch.float32).to(device)
    test_labels = y_test  # numpy array

    out_gru = gru_model(test_t)
    out_cnn = cnn_model(test_cnn)
    final_out = fusion_model(out_gru, out_cnn, test_lgb)
    probs = torch.softmax(final_out, dim=1).cpu().numpy()
    preds = np.argmax(probs, axis=1)
end_infer_time = time.time()
inference_duration = end_infer_time - start_infer_time

# Metrics
acc = accuracy_score(test_labels, preds)
prec = precision_score(test_labels, preds, average='weighted', zero_division=0)
rec = recall_score(test_labels, preds, average='weighted', zero_division=0)
f1 = f1_score(test_labels, preds, average='weighted', zero_division=0)
ll = log_loss(test_labels, probs)

print("\n=== Final Evaluation Metrics ===")
print(f"Accuracy: {acc:.4f}")
print(f"Precision: {prec:.4f}")
print(f"Recall: {rec:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Log Loss: {ll:.4f}")
print(f"Training Time: {train_duration:.2f} seconds")
print(f"Inference Time: {inference_duration:.2f} seconds")

print("\n=== Classification Report ===")
print(classification_report(test_labels, preds, zero_division=0))

Epoch 1, Train Loss: 1.4234, Val Loss: 1.1739
Epoch 2, Train Loss: 0.8694, Val Loss: 0.6062
Epoch 3, Train Loss: 0.3607, Val Loss: 0.2498
Epoch 4, Train Loss: 0.1344, Val Loss: 0.1479
Epoch 5, Train Loss: 0.0591, Val Loss: 0.1188
Epoch 6, Train Loss: 0.0318, Val Loss: 0.1095
Epoch 7, Train Loss: 0.0196, Val Loss: 0.1074
Epoch 8, Train Loss: 0.0133, Val Loss: 0.1074
Epoch 9, Train Loss: 0.0097, Val Loss: 0.1091
Epoch 10, Train Loss: 0.0074, Val Loss: 0.1106
Epoch 11, Train Loss: 0.0058, Val Loss: 0.1124
Epoch 12, Train Loss: 0.0047, Val Loss: 0.1142
Epoch 13, Train Loss: 0.0038, Val Loss: 0.1162
Early stopping triggered.

=== Final Evaluation Metrics ===
Accuracy: 0.9798
Precision: 0.9799
Recall: 0.9798
F1-score: 0.9798
Log Loss: 0.1074
Training Time: 34.14 seconds
Inference Time: 0.08 seconds

=== Classification Report ===
              precision    recall  f1-score   support

           0       0.96      0.96      0.96       138
           1       1.00      0.96      0.98        27
  

In [ ]:
import joblib
joblib.dump(lgb, 'lightgbm_model_updated.pkl')

['lightgbm_model_updated.pkl']

In [ ]:
import json

sample_input = {
    "stat_graph_features": X_statgraph[0].tolist(),  # shape: [N_features]
    "temporal_features": X_temporal[0].reshape(14).tolist()  # shape: [10]
}

with open("sample_input_updated.json", "w") as f:
    json.dump(sample_input, f, indent=2)

In [ ]:
import torch

checkpoint = torch.load("best_model.pth", map_location="cpu")
print("Keys in checkpoint:", checkpoint.keys())

Keys in checkpoint: dict_keys(['gru', 'cnn', 'fusion'])
